In [1]:
#path_name_documents = './Databases/prova/prova4.jsonl'
path_name_documents = './Databases/prova/prova50.jsonl'
#path_name_documents = './Databases/prova/prova30000.jsonl'
#path_name_documents = './Databases/prova/prova2000.jsonl'

In [2]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time:.5f} seconds")
        return result
    return wrapper

In [3]:
import json
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


stop_words = set(stopwords.words('english'))

def stemmingLemming(filtered_tokens):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming or lemmatization on filtered tokens
    
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return filtered_tokens
    
 
    

def tokenize(path_name):
    
    with open(path_name, "r") as f:
        data = f.readlines()

        # Create an empty list to store the tokenized documents
        tokenized_docs = []

        # Loop through each line in the JSONL file
        for line in data:
            # Parse the JSON string into a Python dictionary
            doc = json.loads(line)

            # Extract the text from the dictionary
            text = doc['text']
            text = text.lower()  # Convert to lowercase
            #text = re.sub(r'\d+', '', text)  # Remove all numbers
            text = text.translate(str.maketrans('', '', string.punctuation))  # Remove all punctuation

            # Tokenize the text using NLTK
            tokens = word_tokenize(text)
            tokensStemLem = stemmingLemming(tokens)

            # Add the tokenized document to the list
            tokenized_docs.append(tokensStemLem)

        # Print the tokenized documents
    return tokenized_docs


tokenized_docs = tokenize(path_name_documents)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer


def calculateTFIDF(tokenized_docs):
    
    vectorizer = TfidfVectorizer()
    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names_out()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
    
        

tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(tokenized_docs)

# DA QUI PARALLEL SPARK

### UNA RIGA

In [5]:
def get_row_values_tfidf(tfidf_matrix, doc_id):
    num_docs = tfidf_matrix.shape[0]
    # Estrai la riga specificata dalla matrice TF-IDF come un array densamente popolato
    row_array = tfidf_matrix.toarray()[doc_id]
    # Creazione della lista di coppie chiave-valore
    row_values = [(index, value) for (index, value) in enumerate(row_array) if value != 0.0]
    # Restituisci la lista di coppie chiave-valore
    return doc_id, row_values,num_docs  


doc_id, row_values, num_docs  = get_row_values_tfidf(tfidf_matrix_docs,0)
#row_values

### TUTTE LE RIGHE

In [6]:
def get_all_row_values_tfidf(tfidf_matrix):
    num_docs = tfidf_matrix.shape[0]
    row_values_list = []
    for doc_id in range(num_docs):
        row_array = tfidf_matrix.toarray()[doc_id]
        row_values = [(index, value) for (index, value) in enumerate(row_array) if value != 0.0]
        row_values_list.append((doc_id, row_values))
    return row_values_list


all_rows_values_list = get_all_row_values_tfidf(tfidf_matrix_docs)
#print(all_rows_values_list)

In [7]:
import os
os.environ['PYSPARK_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'


In [8]:
import numpy as np


def sim(list1, list2):
    similarity = abs(list1[0] - list2[0])  # Compute similarity as the absolute difference between the values
    return similarity

def b(document):
    # Example function to calculate the threshold 'b' for a document
    threshold = np.mean([value for _, value in document])
    return threshold



In [9]:
# Funzione di mappatura
def my_map(doc_id, document):
    map_results = []
    threshold = b(document)
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results

#map_results = my_map(doc_id, document)


In [10]:
def execution_map(num_docs,tfidf_matrix):
    map_results
    for i in range(0,num_docs):
        doc_id, row_values, num_docs  = get_row_values_tfidf(tfidf_matrix_docs,i)
        map_results.append(my_map(doc_id, row_values))
    return map_results



### Funzione funzionante per python

In [11]:
from pyspark.sql import SparkSession
import numpy as np

def my_map_python(doc_id, document):
    map_results = []
    threshold = b(document)
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results


def my_reduce_python(map_results, threshold):
    emitted_results = []
    for item in map_results:
        t = item[0]
        doc_id, document = item[1]
        for index, (id1, d1) in enumerate(document):
            for index2 in range(index + 1, len(document)):
                id2, d2 = document[index2]
                if t == max(id1, id2):
                    similarity = sim([d1], [d2])
                    if similarity >= threshold:
                        emitted_results.append((id1, id2, similarity))
    return emitted_results



### Implementazioni per Spark 

In [12]:
#spark = SparkSession.builder.master("local[3]").appName("all-doc-pairs-similarity.com").config("spark.driver.memory", "10g").getOrCreate()

#sc = spark.sparkContext

# -------------------------

In [13]:
datas = get_all_row_values_tfidf(tfidf_matrix_docs)


In [17]:
from pyspark import SparkContext

def sim(list1, list2):
    # Calcola la similarità come la somma delle differenze normalizzate tra i valori delle due liste
    similarity = sum(abs(x - y) for (x, _), (y, _) in zip(list1, list2))
    max_diff = max(x for x, _ in list1)
    similarity = 1 - similarity / max_diff
    return similarity


# Funzione di mapping
def my_map1(doc_id, document,threshold):
    map_results = []
    threshold = 0.2  # Soglia fissa
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results


# Funzione di riduzione
def my_reduce(t, doc_list, threshold):
    emitted_results = []
    for i, doc_item1 in enumerate(doc_list):
        id1, d1 = doc_item1
        for j in range(i + 1, len(doc_list)):
            doc_item2 = doc_list[j]
            id2, d2 = doc_item2
            if id1 != id2:  # Confronta solo gli identificatori
                similarity = sim(d1, d2)
                if similarity >= threshold:
                    emitted_results.append((id1, id2, similarity))
    return emitted_results



# Inizializza lo SparkContext
sc = SparkContext()

threshold = 0.3
# Dati di input
data = [(0, [(2, 0.41137791133379387), (4, 0.6445029922609534), (10, 0.6445029922609534)]),
        (1, [(2, 0.41137791133379387), (3, 0.6445029922609534), (9, 0.6445029922609534)]),
        (2, [(0, 0.5086718718935653), (5, 0.40104274646999605), (6, 0.40104274646999605), (7, 0.5086718718935653), (8, 0.40104274646999605)]),
        (3, [(1, 0.5528163151092931), (2, 0.3528554929793508), (5, 0.43584673254990375), (6, 0.43584673254990375), (8, 0.43584673254990375)])]

# Crea l'RDD di input
input_rdd = sc.parallelize(datas)

# Esegui la funzione di mapping
mapped_rdd = input_rdd.flatMap(lambda x: my_map1(x[0], x[1],threshold))

# Esegui la funzione di riduzione
reduced_rdd = mapped_rdd.groupByKey().flatMap(lambda x: my_reduce(x[0], list(x[1]), 0.2))

# Raccogli i risultati
results = reduced_rdd.collect()

# Elimina duplicati
results = list(set(results))

# Stampa i risultati
for result in results:
    print(result)

# Ferma lo SparkContext
sc.stop()




# --------------------

In [15]:
from pyspark import SparkContext

def sim(list1, list2):
    # Calcola la similarità come la somma delle differenze normalizzate tra i valori delle due liste
    similarity = sum(abs(x - y) for (x, _), (y, _) in zip(list1, list2))
    max_diff = max(x for x, _ in list1)
    similarity = 1 - similarity / max_diff
    return similarity


# Funzione di mapping
def my_map1(doc_id, document,threshold):
    map_results = []
    threshold = 0.2  # Soglia fissa
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results


# Funzione di riduzione
def my_reduce(t, doc_list, threshold):
    emitted_results = []
    for i, doc_item1 in enumerate(doc_list):
        id1, d1 = doc_item1
        for j in range(i + 1, len(doc_list)):
            doc_item2 = doc_list[j]
            id2, d2 = doc_item2
            similarity = sim(d1, d2)
            if similarity >= threshold:
                emitted_results.append((id1, id2, similarity))
    return emitted_results


# Inizializza lo SparkContext
sc = SparkContext()

threshold = 0.3
# Dati di input
data = [(0, [(2, 0.41137791133379387), (4, 0.6445029922609534), (10, 0.6445029922609534)]),
        (1, [(2, 0.41137791133379387), (3, 0.6445029922609534), (9, 0.6445029922609534)]),
        (2, [(0, 0.5086718718935653), (5, 0.40104274646999605), (6, 0.40104274646999605), (7, 0.5086718718935653), (8, 0.40104274646999605)]),
        (3, [(1, 0.5528163151092931), (2, 0.3528554929793508), (5, 0.43584673254990375), (6, 0.43584673254990375), (8, 0.43584673254990375)])]

# Crea l'RDD di input
input_rdd = sc.parallelize(data)

# Esegui la funzione di mapping
mapped_rdd = input_rdd.flatMap(lambda x: my_map1(x[0], x[1],threshold))

# Esegui la funzione di riduzione
reduced_rdd = mapped_rdd.groupByKey().flatMap(lambda x: my_reduce(x[0], list(x[1]), 0.2))

# Raccogli i risultati
results = reduced_rdd.collect()

# Elimina duplicati
results = list(set(results))

# Stampa i risultati
for result in results:
    print(result)

# Ferma lo SparkContext
sc.stop()


(1, 3, 0.33333333333333337)
(0, 1, 0.8)
(2, 3, 0.25)
